In [93]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [94]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [95]:
dataset = pd.read_csv('./dataset.csv')
data = dataset[['Date','Brent',	'WTI',	'USA_Inflation'	,'Closing']].set_index(['Date'])

In [96]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=20, n_out=20, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [97]:
dataset = series_to_supervised(data)

In [98]:
# split into train and test sets
values = dataset.values
n_train_days = 3200
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
X_train, y_train = train[:, :-80], train[:, -80:]
X_test, y_test = test[:, :-80], test[:, -80:]






In [99]:
standarizer = StandardScaler()
standarizer.fit_transform(X_train)
standarizer.transform(y_train)

array([[-1.65631524, -1.64043114, -0.26843944, ..., -1.56713516,
        -0.27050325, -0.39685158],
       [-1.66457744, -1.63284719, -0.26843944, ..., -1.53450699,
        -0.27050325, -0.39685158],
       [-1.68038338, -1.68087883, -0.26843944, ..., -1.54510055,
        -0.27050325, -0.44269024],
       ...,
       [-0.83440613, -0.90858068,  0.27987698, ..., -0.84550178,
         0.41480307,  0.26016928],
       [-0.76759009, -0.86897564,  0.27987698, ..., -0.8327895 ,
         0.41480307,  0.24488972],
       [-0.77621152, -0.85043712,  0.27987698, ..., -0.8327895 ,
         0.41480307,  0.15321239]])

In [100]:
# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0],20, 4))
X_test = X_test.reshape((X_test.shape[0], 20, 4))
y_train = y_train.reshape((y_train.shape[0],20, 4))
y_test = y_test.reshape((y_test.shape[0],20, 4))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3200, 20, 4) (3200, 20, 4) (615, 20, 4) (615, 20, 4)


In [102]:
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(20, 4)))
model.add(RepeatVector(20))
model.add(LSTM(80, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(4)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=20)

Epoch 1/200
3200/3200 [==============================] - 7s 2ms/step - loss: 90200.9223
Epoch 2/200
3200/3200 [==============================] - 5s 2ms/step - loss: 42573.9990
Epoch 3/200
3200/3200 [==============================] - 5s 2ms/step - loss: 7810.1734
Epoch 4/200
3200/3200 [==============================] - 5s 2ms/step - loss: 3095.5987
Epoch 5/200
3200/3200 [==============================] - 5s 2ms/step - loss: 1361.7212
Epoch 6/200
3200/3200 [==============================] - 5s 2ms/step - loss: 868.8022
Epoch 7/200
3200/3200 [==============================] - 5s 2ms/step - loss: 527.6968
Epoch 8/200
3200/3200 [==============================] - 5s 2ms/step - loss: 461.9277
Epoch 9/200
3200/3200 [==============================] - 5s 1ms/step - loss: 394.2433
Epoch 10/200
3200/3200 [==============================] - 5s 1ms/step - loss: 981.3515
Epoch 11/200
3200/3200 [==============================] - 5s 1ms/step - loss: 734.8111
Epoch 12/200
3200/3200 [=====================

3200/3200 [==============================] - 5s 1ms/step - loss: 99.0564
Epoch 189/200
3200/3200 [==============================] - 6s 2ms/step - loss: 107.3066
Epoch 190/200
3200/3200 [==============================] - 5s 2ms/step - loss: 102.5744
Epoch 191/200
3200/3200 [==============================] - 5s 1ms/step - loss: 105.9422
Epoch 192/200
3200/3200 [==============================] - 5s 1ms/step - loss: 99.7626
Epoch 193/200
3200/3200 [==============================] - 5s 1ms/step - loss: 96.7369
Epoch 194/200
3200/3200 [==============================] - 5s 1ms/step - loss: 104.5578
Epoch 195/200
3200/3200 [==============================] - 5s 1ms/step - loss: 99.5416
Epoch 196/200
3200/3200 [==============================] - 5s 1ms/step - loss: 100.0097
Epoch 197/200
3200/3200 [==============================] - 5s 1ms/step - loss: 103.2154
Epoch 198/200
3200/3200 [==============================] - 5s 1ms/step - loss: 99.6111
Epoch 199/200
3200/3200 [==========================

In [ ]:
predict = model.predict(X_test)
predict.reshape(predict.shape[0], 80)

In [ ]:
pred = predict[614].reshape(20,4)

In [ ]:
pd.DataFrame(pred)

In [106]:
test = y_test[614].reshape(20,4)
pd.DataFrame(test)

,0,1,2,3
0,64.07,58.53,1.8,232.0
1,62.90,53.64,1.7,230.0
2,61.12,55.67,1.7,230.0
3,59.32,54.63,1.7,232.0
4,58.63,53.60,1.7,236.0
5,55.03,51.14,1.7,236.0
6,56.29,52.60,1.7,242.0
7,57.37,54.41,1.7,244.0
8,57.13,54.98,1.7,238.0
9,57.86,55.16,1.7,240.0


In [7]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[188.1424  208.91902]]
